# HERA Bioinformatics training

Hi!

Welcome to the hands-on course part of the HERA bioinformatics training.

Before getting started, underneath this text you'll see a button with the text "*Show code*" with a play-button next to it.  
Please click that play button now, it will install and configure everything necessary for this course. 

Installation will take about 5~6 minutes.  
Once a block of code is finished with running, a small green checkmark will appear left next to the play-button.

<u>**Please do not close or refresh the page while going through this course, doing so will cause your progression to be lost.**</u>


---

In [ ]:
#@title
!pip install igv-jupyter --quiet > /dev/null 2>&1
!sed -i -e '1,2d' ~/.bashrc && source ~/.bashrc && bash -c "$(curl -sL https://raw.githubusercontent.com/RIVM-bioinformatics/HERA-Bioinformatics-Training/main/setup.sh)"

---
### About the dataset(s)
The data that is analyzed throughout this course is publicly accessible data downloaded from the ENA (European Nucleotide Archive).  
The data that will be processed are raw FastQ reads both from an Illumina MiSeq sequencer as well as a Nanopore GridIon sequencer.  
The following Illumina dataset will be used:
https://www.ebi.ac.uk/ena/browser/view/ERR4082808?show=reads  
And the following Nanopore dataset will be used: https://www.ebi.ac.uk/ena/browser/view/ERR9900947?show=reads  

### About the analysis steps in this course

The analysis steps in this course are tailored for the analysis of SARS-CoV-2.  
These steps are based on the **SARS2seq** pipeline which is used for SARS-CoV-2 analysis within the RIVM. These various steps *can* be used to analyse other viruses as well, but success is not guaranteed.  




## Table of contents

1. [Removing sequencing adapters & quality control](#scrollTo=O2yVf3BatvPS)
  * [Illumina data](#scrollTo=me1ujCj_gye6)
  * [Nanopore data](#scrollTo=kwlVA4aZjBzZ)
2. [Removing primer sequences](#scrollTo=4mdhoST_S2qp)
  * [Illumina data](#scrollTo=Ny9R7bdhS7BE)
  * [Nanopore data](#scrollTo=JAEWpOp6TJEl)
3. [Aligning reads to reference](#scrollTo=gANzZ3My1hWx)
  * [Illumina data](#scrollTo=QUD__5S41msl)
  * [Nanopore data](#scrollTo=p6tW2tWq1pB9)
4. [Consensus calling](#scrollTo=V7xPm-8zPWDQ)
  * [Illumina data](#scrollTo=i80QZdHtmOVq)
  * [Nanopore data](#scrollTo=mJhqzELmmLav)

# 1. Removing sequencing adapters & quality control

## Illumina data

### Removing adapters from Illumina Sequencing data
This step is usally done with tools such as [Trimmomatic](https://github.com/usadellab/Trimmomatic) or [FastP](https://github.com/OpenGene/fastp).  
Both Trimmomatic as well as FastP are read-cleaning tools which can be used to efficiently clean several aspects of your FastQ reads. For Illumina data, these tools also include a function to remove sequencing adapters with very high accuracy.

Using Trimmomatic or Fastp for this step should be your go-to solution.
However, here we will perform this step with a very different method.  

SARS-CoV-2 or other reference based viral analysis has the benefit that during analysis we already have the knowledge of what the end result should (somewhat) look like. Therefore we are able to remove sequencing adapters by aligning the reads to the reference with a read-aligner such as Minimap2.  
This particular method is many times more resource efficient when doing a 'target analysis' as we are doing here.

Please also see the guiding presentation where we explain more how this works.

Here we're using Minimap2 as a read-aligner. This is an aligner that works very well for a virus such as SARS-CoV-2. If you want to use this method for other viruses with Illumina data then please note you may need to use another read-aligner such as Bowtie2 or BWA-MEM.

In [ ]:
%%bash
source activate base; conda activate Alignments

## Align the reads to the reference sequence
minimap2 -ax sr source/extra/GCF_009858895_2_ASM985889v3_genomic.fasta example_data/illumina_fastq_1.fastq.gz example_data/illumina_fastq_2.fastq.gz | samtools view -F 256 -F 512 -F 4 -F 2048 -uS | samtools sort -o output_data/alignments/illumina_raw_alignment.bam
samtools index output_data/alignments/illumina_raw_alignment.bam

## extract the reads from the alignment back to a single fastq file without the adapters
python source/extra/clipper.py --input output_data/alignments/illumina_raw_alignment.bam --output output_data/adapter_removal/illumina_no_adapters.fastq

### Quality control in Illumina data

When adapters have been removed from the reads it is necessary to perform quality control and data cleaning before continuing.  This is necessary to remove, for example, uncertain nucleotide calls caused by homopolymer regions or to remove reads from the dataset which are too short.  
Additionally, when using Illumina sequencers the certainty of nucleotide calls degrades when approaching the end of a read.

Usually the Quality Control and data cleaning can be combined with adapter removal if a tool such as FastP or Trimmomatic is used.

### Questions
* What is a PHRED score?
* Why do we only cut the right side of the read?
* Is this the 3' or 5' end
* What does the window size and -l parameter specify?

In [ ]:
%%bash
source activate base; conda activate Data_cleanup

fastp -i output_data/adapter_removal/illumina_no_adapters.fastq -o output_data/quality_control/illumina_post_qc.fastq \
  -A --cut_right --cut_right_mean_quality 20 --cut_right_window_size 5 -l 100 \
  -h output_data/quality_control/illumina_fastp.html -j output_data/quality_control/illumina_fastp.json

In [ ]:
#@markdown <-- click to show quality control report
!sed -i 's/http:\/\//https:\/\//g' ./output_data/quality_control/*.html
from IPython.display import HTML

HTML(filename="/content/output_data/quality_control/illumina_fastp.html")

## Nanopore data

### Removing sequencing adapters from Nanopore Sequencing data
Just like with Illumina data, sequencing adapters need to be removed from the Nanopore dataset.  
With Nanopore data this can be done with a tool such as [Porechop](https://github.com/rrwick/Porechop), but it's also possible to perform this step with the default basecalling tool (Guppy) provided by Nanopore themselves.  

Because of the error-rate present in Nanopore sequencing reads these tools are unable to clean the reads with 100% accuracy. It's therefore advised to check wether your adapters were properly removed by the cleaning tool to make sure this won't cause a problem later on.

As we also did with the Illumina data, we won't use either Porechop or Guppy here to remove the adapters. Instead we will again use the aligner Minimap2 as a means to remove the adapters since we're doing a targeted analysis.

In [ ]:
%%bash
source activate base; conda activate Alignments

## Align the reads to the reference sequence
minimap2 -ax map-ont source/extra/GCF_009858895_2_ASM985889v3_genomic.fasta example_data/nanopore_fastq.fastq.gz | samtools view -F 256 -F 512 -F 4 -F 2048 -uS | samtools sort -o output_data/alignments/nanopore_raw_alignment.bam
samtools index output_data/alignments/nanopore_raw_alignment.bam

## Extract the reads from the alignment back to a single fastq file without the adapters
python source/extra/clipper.py --input output_data/alignments/nanopore_raw_alignment.bam --output output_data/adapter_removal/nanopore_no_adapters.fastq

### Quality control in Nanopore data

The best filtering tool depends on your experimental setup as well as the data which you have available.  
There are many nanopore-specific filtering tools such as [NanoFilt](https://github.com/wdecoster/nanofilt) or [nanopolish](https://github.com/jts/nanopolish).   Tools such as nanopolish rely on unprocessed raw data (fast5 data) from the Nanopore sequencer to perform its tasks.
Depending on your analysis-setup this data may not always be directly available, such is the case here as we're just using FastQ data.   
 
Because of this we don't need anything 'nanopore-specific' for this step, we can therefore use a tool like FastP which can be used in a more generic data-cleaning sense.

### Questions
* What differences do you see when comparing the quality reports of the Nanopore and Illumina reads?
* What does the -l parameter do?

In [ ]:
%%bash
source activate base; conda activate Data_cleanup

fastp -i output_data/adapter_removal/nanopore_no_adapters.fastq -o output_data/quality_control/nanopore_post_qc.fastq \
  -A --cut_right --cut_right_mean_quality 7 --cut_right_window_size 20 -l 100 \
  -h output_data/quality_control/nanopore_fastp.html -j output_data/quality_control/nanopore_fastp.json

In [ ]:
#@markdown <-- click to show quality control report
!sed -i 's/http:\/\//https:\/\//g' ./output_data/quality_control/*.html
from IPython.display import HTML

HTML(filename="/content/output_data/quality_control/nanopore_fastp.html")

# 2. Removing Primer sequences
After performing quality filtering and trimming with FastP (or another tool) it's necessary to remove the primer sequences from the reads.  As with the earlier steps, there are various tools that can be used for this. For example the tool [CutAdapt](https://github.com/marcelm/cutadapt). 

However we use the tool [AmpliGone](https://rivm-bioinformatics.github.io/AmpliGone/1.1.0/) for this step as it is a tool which is developed exactly for the experimental setup and situation which is presented in the data that we're analysing here.
AmpliGone can be used for both Illumina and Nanopore data.

* What do you risk if you do not remove primers before determining a consensus sequence?
* Look at the BED file. What does it describe?

## Illumina data

* Does the end-to-mid protocol cut primers from the 5' end?
* Does the end-to-mid protocol cut primers from the 3' end?

In [ ]:
%%bash
source activate base; conda activate Data_cleanup

AmpliGone --input output_data/quality_control/illumina_post_qc.fastq \
  --output output_data/primer_removal/illumina_no_primers.fastq \
  --reference source/extra/GCF_009858895_2_ASM985889v3_genomic.fasta \
  --primers source/extra/articv3.bed \
  --amplicon-type end-to-mid \
  --export-primers output_data/primer_removal/illumina_removed_primers.bed

## Nanopore data

* Does the end-to-end protocol cut primers from the 5' end?
* Does the end-to-end protocol cut primers from the 3' end?


In [ ]:
%%bash
source activate base; conda activate Data_cleanup

AmpliGone --input output_data/quality_control/nanopore_post_qc.fastq \
  --output output_data/primer_removal/nanopore_no_primers.fastq \
  --reference source/extra/GCF_009858895_2_ASM985889v3_genomic.fasta \
  --primers source/extra/articv3.bed \
  --amplicon-type end-to-end \
  --export-primers output_data/primer_removal/nanopore_removed_primers.bed

# 3. Aligning reads to a reference

Once your data is properly cleaned and primers are removed you can align the reads to the SARS-CoV-2 reference sequence. This is again done with a read-aligner.  
For SARS-CoV-2 analysis we use Minimap2 for both Illumina and Nanopore data as this results in good quality alignments for both sequencing platforms with this particular virus.  
With Minimap2 only the alignment preset needs to be adjusted for compatibility with the corresponding sequencing platform.

Please note that you may need to use a different read-aligner for Illumina data depending on the virus and preprocessing steps that have been performed.


The tool Samtools is then used to filter the output generated (for example discard reads that couldn't be aligned to the reference) by the read-aligner and to also write an output file in the correct format for subsequent tasks.

Once alignment is done you can view the read-alignment results interactively.

## Illumina data

* Do you see any missing amplicons?
* If yes, which ones?

In [ ]:
%%bash
source activate base; conda activate Alignments

## Align the reads to the reference sequence
minimap2 -ax sr source/extra/GCF_009858895_2_ASM985889v3_genomic.fasta output_data/primer_removal/illumina_no_primers.fastq | samtools view -F 256 -F 512 -F 4 -F 2048 -uS | samtools sort -o output_data/alignments/illumina_cleaned_alignment.bam
samtools index output_data/alignments/illumina_cleaned_alignment.bam

In [ ]:
#@markdown <-- Click to show alignment results

!source activate base; conda activate Alignments; samtools view -bs 0.05 /content/output_data/alignments/illumina_cleaned_alignment.bam > /content/output_data/alignments/illumina_cleaned_subsampled_for_view.bam
!source activate base; conda activate Alignments; samtools index /content/output_data/alignments/illumina_cleaned_subsampled_for_view.bam
import igv_notebook
igv_notebook.init()
b = igv_notebook.Browser(
    {
        "genome": "ASM985889v3",
        "locus": "NC_045512.2:1-300",
        "tracks": [
          {
            "name": "Illumina read alignment",
            "path": "/content/output_data/alignments/illumina_cleaned_subsampled_for_view.bam",
            "indexPath": "/content/output_data/alignments/illumina_cleaned_subsampled_for_view.bam.bai",
            "type": "alignment",
            "format": "bam",
            "showSoftClips": False,
            "colorBy": "strand"
           },
           {
              "name": "Removed primers",
              "type": "annotation",
              "format": "bed",
              "path": "/content/output_data/primer_removal/illumina_removed_primers.bed",
              "displayMode": "EXPANDED"
           }
        ]
    }
)

## Nanopore data

* Do you see any mutations in positions where primers bind?
* Does this seem to effect the coverage for the associated amplicon?
* Can you find reads that are longer than the typical amplicon length? Is this a problem?

In [ ]:
%%bash
source activate base; conda activate Alignments

## Align the reads to the reference sequence
minimap2 -ax map-ont source/extra/GCF_009858895_2_ASM985889v3_genomic.fasta output_data/primer_removal/nanopore_no_primers.fastq | samtools view -F 256 -F 512 -F 4 -F 2048 -uS | samtools sort -o output_data/alignments/nanopore_cleaned_alignment.bam
samtools index output_data/alignments/nanopore_cleaned_alignment.bam

In [ ]:
#@markdown <-- Click to show alignment results

!source activate base; conda activate Alignments; samtools view -bs 0.05 /content/output_data/alignments/nanopore_cleaned_alignment.bam > /content/output_data/alignments/nanopore_cleaned_subsampled_for_view.bam
!source activate base; conda activate Alignments; samtools index /content/output_data/alignments/nanopore_cleaned_subsampled_for_view.bam
import igv_notebook
igv_notebook.init()
b = igv_notebook.Browser(
    {
        "genome": "ASM985889v3",
        "locus": "NC_045512.2:1-300",
        "tracks": [
          {
            "name": "Nanopore read alignment",
            "path": "/content/output_data/alignments/nanopore_cleaned_subsampled_for_view.bam",
            "indexPath": "/content/output_data/alignments/nanopore_cleaned_subsampled_for_view.bam.bai",
            "type": "alignment",
            "format": "bam",
            "showSoftClips": False,
            "colorBy": "strand"
           },
           {
              "name": "Removed primers",
              "type": "annotation",
              "format": "bed",
              "path": "/content/output_data/primer_removal/nanopore_removed_primers.bed",
              "displayMode": "EXPANDED"
           }
        ]
    }
)

# 4. Consensus calling

When the reads have been aligned and a BAM (Binary Alignment Map) file is present we can start with the process of creating a consensus sequence.

There are different ways to create a consensus sequence. The most common method is to create a VCF (Variant Call Format) file first.  
A VCF file is made by with a variant caller such as [LoFreq](https://github.com/CSB5/lofreq) or [BCFtools](https://github.com/samtools/bcftools).  
There is no such thing as a 'best variant caller' as this is highly dependant on the data and experimental setup that you're processing.  


Another method is to not use a variant caller at all and make a consensus sequence directly from the earlier generated BAM file. This is also what we will be doing below.  
The tool we will be using for this is [TrueConsense](https://github.com/RIVM-bioinformatics/TrueConsense) which is a consensus-caller for viral-targets.  
The benefit of making a consensus-genome directly from a BAM file is that a lot of contextual information is still present. This contextual information can quickly get lost when using an intermediate file containing only differences.

One method is not necessarily better than another method, this depends on your own data, the complexity of the pathogen itself as well as the experimental setup that was used.

## Illumina data

* What do think is a good minimum coverage level to use for illumina data?

In [ ]:
%%bash
source activate base; conda activate Consensus_seq

TrueConsense --input output_data/alignments/illumina_cleaned_alignment.bam \
  --reference /content/source/extra/GCF_009858895_2_ASM985889v3_genomic.fasta \
  --features /content/source/extra/features.gff \
  --coverage-level 10 \
  --samplename illumina_example \
  --output output_data/illumina_consensus_sequence.fasta \
  --variants output_data/illumina_variants.vcf \
  --output-gff output_data/illumina_corrected_features.gff \
  --depth-of-coverage output_data/illumina_coverage.tsv

## Nanopore data

* What do think is a good minimum coverage level to use for nanopore data?
* Inspect the vcf and gff files. Can you find (some of) the mutations in the gff back in the alignment?
* Which clade is this sequence according to NextClade?
* Which lineage is this sequence according to Pangolin?

In [ ]:
%%bash
source activate base; conda activate Consensus_seq

TrueConsense --input output_data/alignments/nanopore_cleaned_alignment.bam \
  --reference /content/source/extra/GCF_009858895_2_ASM985889v3_genomic.fasta \
  --features /content/source/extra/features.gff \
  --coverage-level 30 \
  --samplename nanopore_example \
  --output output_data/nanopore_consensus_sequence.fasta \
  --variants output_data/nanopore_variants.vcf \
  --output-gff output_data/nanopore_corrected_features.gff \
  --depth-of-coverage output_data/nanopore_coverage.tsv

# 5. Manual curation

Now that the consensus sequence is generated from both the Illumina and Nanopore data we have to perform a manual curation step to see if everything that we have done resulted in a proper consensus genome.  

You can download the generated consensus sequences from this environment to your own computer and use a web-based tool such as [NextClade](https://clades.nextstrain.org/) to see how the generated consensus genome turned out.